In [1]:
import numpy as np
import pandas as pd
import math
np.set_printoptions(suppress=True)

In [2]:
def check_orthogonal(rot_mat):
    print("Orthogonality Check for Rotation Matrix")
    for i in range(3):
        print((np.sum(np.power(rot_mat[i],2))))

ZYX Euler Angle Rotation Matrix

In [3]:
def rot_mat_create(mu, sigma):
  # Euler angle을 이용한 3x3 임의회전행렬 생성
    alpha, beta, gamma = np.random.normal(mu, sigma, 3)  # 정규분포 내에서 표본 크기가 3인 임의표본 생성
    print("alpha, beta, gamma", alpha, beta, gamma)

    rz = [[np.cos(alpha),-np.sin(alpha),0],[np.sin(alpha),np.cos(alpha),0],[0,0,1]]
    ry = [[np.cos(beta),0,np.sin(beta)],[0,1,0],[-np.sin(beta),0,np.cos(beta)]]
    rx = [[1,0,0],[0,np.cos(gamma),-np.sin(gamma)],[0,np.sin(gamma),np.cos(gamma)]]
    # alpha, beta, gamma를 대입한 회전행렬 생성

    rot_mat = np.dot(np.dot(rz,ry),rx)
    # 직교성 검증을 위한 내적 계산
    # check_orthogonal(rot_mat)

    return rot_mat

In [4]:
def trans_mat_create(mu, sigma, translation):
    # 4x4 임의변환행렬 생성
    alpha, beta, gamma = np.random.normal(mu, sigma, 3)

    # print(alpha,beta,gamma)
    rz = [[np.cos(alpha),-np.sin(alpha),0],[np.sin(alpha),np.cos(alpha),0],[0,0,1]]
    ry = [[np.cos(beta),0,np.sin(beta)],[0,1,0],[-np.sin(beta),0,np.cos(beta)]]
    rx = [[1,0,0],[0,np.cos(gamma),-np.sin(gamma)],[0,np.sin(gamma),np.cos(gamma)]]
    # 여기까지는 회전행렬(3x3)

    rot_mat = np.dot(np.dot(rz,ry),rx)
    # check_orthogonal(rot_mat)

    rot_mat = np.vstack([rot_mat, [0, 0, 0]])
    print(rot_mat)
    # 변환행렬 생성
    trans_mat = np.hstack([rot_mat, np.array(translation).reshape(-1, 1)])
    print(trans_mat)

    return trans_mat

로드리게스(샌딩 머신 회전 행렬)

In [5]:
def K(n):
    nx = n[0]
    ny = n[1]
    nz = n[2]
    return np.array([[0, nz, -ny],[-nz, 0, nx],[ny, -nx, 0]])

def rod_rot(n,theta):
    kn = K(n)
    rod_rot = np.array([[1,0,0], [0,1,0], [0,0,1]]) + np.sin(theta)*kn + (1-np.cos(theta))*np.dot(kn,kn)
    return rod_rot

In [6]:
#random vector
mu = 9
sigma = 1000
vec1 = np.random.randint(-200, 200, 3)
rot_mat = rot_mat_create(90, 100000)
vec2 = np.dot(vec1,rot_mat)
print("Two vectors", vec1, vec2)
check_orthogonal(rot_mat)
print("vector size", np.linalg.norm(vec1), np.linalg.norm(vec2))

angle = np.arccos((np.dot(vec1,vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2))))
per_vec = np.cross(vec1, vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2)*np.sin(angle))

rodrigues = rod_rot(per_vec, angle)
print("vec2 by original and rodrigues\n Original:", vec2, "\n Rodrigues:",np.dot(vec1, rodrigues))

alpha, beta, gamma -130575.42468429345 -91211.88042603413 -110203.04249805334
Two vectors [192 117  82] [ -37.03021725   45.09563143 -232.10374197]
Orthogonality Check for Rotation Matrix
1.0
0.9999999999999999
0.9999999999999999
vector size 239.32613731057458 239.32613731057458
vec2 by original and rodrigues
 Original: [ -37.03021725   45.09563143 -232.10374197] 
 Rodrigues: [ -37.03021725   45.09563143 -232.10374197]


ZYX 각각 & ZYX 통채로 비교

In [7]:
alpha,beta,gamma = np.random.normal(mu, sigma, 3)
#print("alpha, beta, gamma", alpha,beta,gamma)

rz = [[np.cos(alpha),-np.sin(alpha),0],[np.sin(alpha),np.cos(alpha),0],[0,0,1]]
ry = [[np.cos(beta),0,np.sin(beta)],[0,1,0],[-np.sin(beta),0,np.cos(beta)]]
rx = [[1,0,0],[0,np.cos(gamma),-np.sin(gamma)],[0,np.sin(gamma),np.cos(gamma)]]
#create rotation matrix
rot_mat = np.dot(np.dot(rz,ry),rx)
#check_orthogonal(rot_mat)

sa = np.sin(alpha)
sb = np.sin(beta)
sr = np.sin(gamma)

ca = np.cos(alpha)
cb = np.cos(beta)
cr = np.cos(gamma)

rot_mat2 = np.array([[ca*cb,ca*sb*sr-sa*cr,ca*sb*cr+sa*sr],
                     [sa*cb,sa*sb*sr+ca*cr,sa*sb*cr-ca*sr],
                     [-sb,cb*sr,cb*cr]
                    ])

print("orig mat:\n", rot_mat)
print("second mat:\n", rot_mat2)

orig mat:
 [[ 0.69460702  0.707899    0.12806287]
 [ 0.15853853  0.0230074  -0.98708469]
 [-0.70170266  0.70593886 -0.09624816]]
second mat:
 [[ 0.69460702  0.707899    0.12806287]
 [ 0.15853853  0.0230074  -0.98708469]
 [-0.70170266  0.70593886 -0.09624816]]


Rx, Ry, Rz(Angle Calculation) Function for ZYX Euler Angle Rotation Matrix

In [8]:
def zyx_angle(rot_mat):
    r11 = rot_mat[0][0]
    r12 = rot_mat[0][1]
    r21 = rot_mat[1][0]
    r22 = rot_mat[1][1]
    r31 = rot_mat[2][0]
    r32 = rot_mat[2][1]
    r33 = rot_mat[2][2]

    stand = r11**2+r21**2
    if (r31 != 1 and r31 != -1):
        beta = math.atan2(-r31,np.sqrt(stand))
        alpha = math.atan2(r21,r11)
        gamma = math.atan2(r32,r33)
    elif r31 == 1:
        beta = -np.pi/2
        alpha = 0
        gamma = -math.atan2(r12,r22)
    else:#r31 == -1
        beta = np.pi/2
        alpha = 0
        gamma = math.atan2(r12,r22)
    return alpha, beta, gamma

In [9]:
# Inverse Angle Calculation Check

alpha,beta,gamma = np.random.normal(mu, sigma, 3)%(2*np.pi)
beta = np.pi/2
print("alpha, beta, gamma:\n", alpha,beta,gamma)

rz = [[np.cos(alpha),-np.sin(alpha),0],[np.sin(alpha),np.cos(alpha),0],[0,0,1]]
ry = [[np.cos(beta),0,np.sin(beta)],[0,1,0],[-np.sin(beta),0,np.cos(beta)]]
rx = [[1,0,0],[0,np.cos(gamma),-np.sin(gamma)],[0,np.sin(gamma),np.cos(gamma)]]
#create rotation matrix
rot_mat = np.dot(np.dot(rz,ry),rx)

print("Angle Inverse Calculation:\n", zyx_angle(rot_mat),"\n")

alpha, beta, gamma = zyx_angle(rot_mat)
rz = [[np.cos(alpha),-np.sin(alpha),0],[np.sin(alpha),np.cos(alpha),0],[0,0,1]]
ry = [[np.cos(beta),0,np.sin(beta)],[0,1,0],[-np.sin(beta),0,np.cos(beta)]]
rx = [[1,0,0],[0,np.cos(gamma),-np.sin(gamma)],[0,np.sin(gamma),np.cos(gamma)]]

rot_mat2 = np.dot(np.dot(rz,ry),rx)

print("Original matrix\n", rot_mat,"\n")
print("Inverse calculated original matrix\n", rot_mat2)

alpha, beta, gamma:
 5.9898628548085 1.5707963267948966 0.8809340518256121
Angle Inverse Calculation:
 (0, 1.5707963267948966, 1.1742565041966986) 

Original matrix
 [[ 0.          0.92240293  0.38622898]
 [-0.          0.38622898 -0.92240293]
 [-1.          0.          0.        ]] 

Inverse calculated original matrix
 [[ 0.          0.92240293  0.38622898]
 [ 0.          0.38622898 -0.92240293]
 [-1.          0.          0.        ]]
